
<b><FONT size="6" color='#884846'>Projet Python Avance </FONT>

<b><FONT size="4" color='#595949'>Fancello Marie Clara, Germini Eva, Gutfreund Eloise

<b><FONT size="5" color='#884846'>Web scraping </FONT>

<b><FONT size="4" color='#858846'>Importation des librairies</FONT>

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Import pour le webscaping 
from lxml import html
import requests
from unidecode import unidecode
import unicodedata
from bs4 import BeautifulSoup
import re
import urllib
from IPython.display import Image 

# Pour le modèle logit  : 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Pour le modèle random forest : 
from sklearn.ensemble import RandomForestClassifier

# Pour le score 
from sklearn.metrics import accuracy_score

<b><FONT size="4" color='#858846'>Recuperation des URL </FONT>

Nous avons choisi de traiter les bijoux pour femmes de chez histoire d'or. 

In [2]:
#Liste vide d'URL
article_url = []
#Initialisation de la boucle 
start = 0

for start in range(0, 3000, 26):
    start = start+1
    url = f"https://www.histoiredor.com/on/demandware.store/Sites-HO-COM-Site/fr_FR/Search-UpdateGrid?cgid=20296&prefn1=THOM-genre&prefv1=Femme&start={start}&sz=24"
    soup = BeautifulSoup(urllib.request.urlopen(url))
     #on récupère des url
    for a in soup.find_all('a', class_="c-product-tile__name-link"): 
        article_url.append(a['href'])

In [3]:
#Taille de la liste 
len(article_url)

2784

<b><FONT size="4" color='#858846'>Carateristiques des produits </FONT>

In [4]:
#Listes vides regroupant les informations par articles 
liste_descp = []
liste_titre = []
liste_prix = []

for url in article_url:
    page = requests.get("https://www.histoiredor.com"+url) #ouvrir l'url 
    soup = BeautifulSoup(page.content, "html.parser")
    tree = html.fromstring(page.content)
    
    #Titre de l'article
    titre = tree.xpath('//div[@data-component="product-name"]/text()')
    liste_titre.append(titre)
    
    #Prix de l'article
    prix = soup.find("span", class_="c-price__standard")
    a = prix['content']
    liste_prix.append(a)
    
    #Description détaillee de l'article 
    nom = soup.find_all("span",class_="c-product__attributes__label")
    value = soup.find_all("span",class_="c-product__attributes__value")
    dictionnaire = dict(zip(nom, value))
    string = str(dictionnaire)
    string = re.sub(r'(\n)', '', string)
    liste_descp.append(string)

In [5]:
#Creation d'une base de donnees a partir des elements "parses"
df_article = pd.DataFrame(list(zip(liste_titre,liste_prix,liste_descp)),
                               columns = ['article', 'prix','description'])

In [6]:
#Taille base 
df_article.tail()

,article,prix,description
2779,[\nBague Anja Or Blanc Rubis Et Diamant\n],360.00,"{<span class=""c-product__attributes__label"">Ge..."
2780,[\nBague Solitaire Victoria Or Blanc Diamant\n],590.00,"{<span class=""c-product__attributes__label"">Ge..."
2781,[\nCréoles Telya Lisses Fil Carre Or Jaune\n],99.00,"{<span class=""c-product__attributes__label"">Ge..."
2782,[\nBague Leni Or Blanc Diamant\n],319.00,"{<span class=""c-product__attributes__label"">Ge..."
2783,[\nAlliance Valentine Or Blanc Diamant Synthet...,790.00,"{<span class=""c-product__attributes__label"">Ge..."


In [7]:
#Exportation de la base de donnees 
df_article.to_csv(r'C:\Users\evage\Desktop\PYTHON\projet\data.csv',index=False, sep=";")

<b><FONT size="5" color='#884846'>Modele avec python </FONT>

Dans cette partie nous reprenons les bases de donnees train et test traitees avec Pyspark sur Databricks.

Nous allons effectuer les memes modeles que ceux effectues avec Pyspark afin de le comparer.

<b><FONT size="4" color='#858846'>Importation des bases de donnees</FONT>

In [8]:
train = pd.read_csv(r'C:\Users\evage\Desktop\PYTHON\projet\train.csv')
test = pd.read_csv(r'C:\Users\evage\Desktop\PYTHON\projet\test.csv')

In [11]:
#Formation des bases de design 
X_train = train.drop(columns="prix_cat")
X_test = test.drop(columns="prix_cat")

#Formation de la variable explicative 
y_train = train["prix_cat"]
y_test = test["prix_cat"]

<b><FONT size="4" color='#858846'> One-Hot-Encoding </FONT>

In [12]:
#Listes des variables de la matrice de design 
var_model = X_train.columns

#One-Hot-Encoding
X_train= pd.get_dummies(X_train, columns=var_model)
X_test = pd.get_dummies(X_test, columns=var_model) 

In [13]:
#Cross validation 
cv= KFold(n_splits=5, shuffle=True, random_state=42)

<b><FONT size="4" color='#858846'> Modele logit </FONT>

In [16]:
#Modele classifier 
model = LogisticRegression(solver = 'saga', penalty='elasticnet')

#Parametres de la GridSearchCV
parametres = {'max_iter':  [10, 50, 100],  
              'l1_ratio': [0.01, 0.1, 0.3]}

#GridSearchCV
clf_cv = GridSearchCV(model,param_grid=parametres, cv=cv, n_jobs=5, scoring='accuracy')

#Entrainement du modele 
clf_cv.fit(X_train, y_train)

#Resultats de la GridSearchCV
print("Le meilleur modèle est : ", clf_cv.best_estimator_)
print("L'accuracy est de : ", clf_cv.best_score_)
print("Les meilleurs paramètres sont : ", clf_cv.best_params_)

#Prediction 
y_pred=clf_cv.predict(X_test)
#Score final sur notre base de test 
print("L'accuracy sur le test est de : ", accuracy_score(y_test, y_pred))

Le meilleur modèle est :  LogisticRegression(l1_ratio=0.01, max_iter=10, penalty='elasticnet',
                   solver='saga')
L'accuracy est de :  0.6447769028871392
Les meilleurs paramètres sont :  {'l1_ratio': 0.01, 'max_iter': 10}
L'accuracy sur le test est de :  0.665948275862069


Le modele logit donne une performance de 66%. Ce qui signifie qu'il ne retrouve pas la bonne classe dans 34% des cas. Ce taux d'erreur est tout de même important.

<b><FONT size="4" color='#858846'> Modele Random Forest </FONT>

In [18]:
#Modele Classifier 
model_RF=RandomForestClassifier()

#Parametres de la GridSearchCV 
parametres = {'max_depth':  [10, 50, 100, 150],
 'n_estimators': [10, 50, 100]}

#GridSearchCV 
clf_rf_cv = GridSearchCV(model_RF,param_grid=parametres, cv=cv, n_jobs=5,  scoring='accuracy')

#Entrainement du modele 
clf_rf_cv.fit(X_train, y_train)

#Resultats de la GridSearchCV 
print("Le meilleur modèle est : ", clf_rf_cv.best_estimator_)
print("L'accuracy est de : ", clf_rf_cv.best_score_)
print("es meilleurs paramètres sont : ", clf_rf_cv.best_params_)

#Predictions 
y_pred=clf_rf_cv.predict(X_test)
#Score final sur notre base de test 
print("L'accuracy sur le test est de : ", accuracy_score(y_test, y_pred))

Le meilleur modèle est :  RandomForestClassifier(max_depth=10)
L'accuracy est de :  0.6946898742920293
es meilleurs paramètres sont :  {'max_depth': 10, 'n_estimators': 100}
L'accuracy sur le test est de :  0.6681034482758621


Le modele random forest ne performe pas beaucoup mieux que le modele logit. Il ne trouve pas la bonne classe de prix dans 33% des cas. 

<b><FONT size="5" color='red'>Conclusion </FONT>

- Au sein de ce notebook, nous avons web-scape le site Histoire d'Or afin d'obtenir les caracteristiques de chaque article feminin. 
- Ensuite, au sein du notebook de databricks, nous avons cree la base de donnees, nettoye la base de donnees et procede a des analyses descriptives. Par extension, nous avons propose deux modeles de Machine Learning afin de predire les categories de prix des produits. Le modele logit et la random forest avaient respectivement une accuracy de 63% et 70%. 
- Nous avons egalement execute ces memes modeles avec la librairie sklearn de python. Nous constatons que le modele de classification logit a une meilleure performance avec la librairie sklearn (accuracy de 66%). Cependant, la random forest a une capacite predictive inferieure a celle effectuee sur pyspark (67% VS 70%). 

- En conclusion, le modele qui permet de predire la categorie de prix des articles avec une erreur minimale est celui de la random forest de pyspark. 